In [1]:
import numpy as np

### Instantiate the prediction task

In [2]:
from jenga.tasks.income import IncomeEstimationTask

task = IncomeEstimationTask(seed=42)

### Task details

In this task, we try to predict the income level of a person (more or less than 50K dollars per years) from demographic and work-related data. It is often used as a proxy to study automated decision making for loan applications.

The original data is available in the UCI machine learning repository at https://archive.ics.uci.edu/ml/datasets/adult

In [3]:
task.train_data

,workclass,occupation,marital_status,education,hours_per_week,age
18907,Private,Exec-managerial,Divorced,Bachelors,47,48
10463,Private,Adm-clerical,Divorced,Some-college,37,41
14837,Private,Tech-support,Never-married,Some-college,50,21
4543,Private,Farming-fishing,Married-civ-spouse,HS-grad,60,34
25174,Private,Craft-repair,Married-civ-spouse,HS-grad,40,50
...,...,...,...,...,...,...
1534,Private,Other-service,Divorced,Some-college,40,50
28946,Private,Prof-specialty,Married-civ-spouse,Masters,40,56
8757,NaN,NaN,Never-married,Some-college,16,22
1608,Private,Craft-repair,Married-civ-spouse,10th,40,39


### Train the provided baseline model

Jenga allows us to easily train and evaluate a logistic regression model for this task. Have a look at https://github.com/schelterlabs/jenga/blob/master/jenga/tasks/income.py if you want to know the details.


In [4]:
model = task.fit_baseline_model()

In [5]:
f"The ROC AUC score on the test data is {task.get_baseline_performance()}"

'The ROC AUC score on the test data is 0.8732229444875153'

### Let's have a look at the test data

In [6]:
task.test_data

,workclass,occupation,marital_status,education,hours_per_week,age
16732,Private,Exec-managerial,Divorced,Bachelors,40,32
4116,Private,Transport-moving,Married-civ-spouse,Assoc-voc,20,65
21650,Private,Craft-repair,Never-married,HS-grad,65,36
10393,Self-emp-not-inc,Prof-specialty,Never-married,Prof-school,40,47
7557,Self-emp-not-inc,Sales,Married-civ-spouse,HS-grad,50,28
...,...,...,...,...,...,...
9119,Private,Sales,Divorced,Masters,40,28
19487,Private,Machine-op-inspct,Married-civ-spouse,HS-grad,30,29
13247,Private,Adm-clerical,Divorced,Some-college,40,44
18701,Local-gov,Prof-specialty,Never-married,Bachelors,15,34


### Corruptions

Jenga provides a set of predefined data corruptions that we can use to simulate errors in the test data. We will simulate 'implicit missing values' in a column and see how this impacts the prediction quality.

Jenga supports much more predefined corruptions, have a look at https://github.com/schelterlabs/jenga/tree/master/jenga/corruptions.


In [7]:
from jenga.corruptions.generic import MissingValues

marital_status_corruption = MissingValues(column='marital_status', fraction=0.99, na_value='BROKEN')

In [8]:
corrupted_test_data = marital_status_corruption.transform(task.test_data)
corrupted_test_data

,workclass,occupation,marital_status,education,hours_per_week,age
16732,Private,Exec-managerial,BROKEN,Bachelors,40,32
4116,Private,Transport-moving,BROKEN,Assoc-voc,20,65
21650,Private,Craft-repair,BROKEN,HS-grad,65,36
10393,Self-emp-not-inc,Prof-specialty,BROKEN,Prof-school,40,47
7557,Self-emp-not-inc,Sales,BROKEN,HS-grad,50,28
...,...,...,...,...,...,...
9119,Private,Sales,BROKEN,Masters,40,28
19487,Private,Machine-op-inspct,BROKEN,HS-grad,30,29
13247,Private,Adm-clerical,BROKEN,Some-college,40,44
18701,Local-gov,Prof-specialty,BROKEN,Bachelors,15,34


In [9]:
y_pred = model.predict_proba(corrupted_test_data)

f"The ROC AUC score on the corrupted test data is {task.score_on_test_data(y_pred)}"

'The ROC AUC score on the corrupted test data is 0.8035382971359658'

### Jenga's evaluators

Jenga provides a set of evaluators which allow us to automate the evaluation of the impact of given data corruptions.

In [10]:
from jenga.evaluation.corruption_impact import CorruptionImpactEvaluator

evaluator = CorruptionImpactEvaluator(task)

In [11]:
corruptions = [
    MissingValues(column='marital_status', fraction=0.99, na_value='BROKEN'),
    MissingValues(column='age', fraction=0.05, na_value=-999),
]

### Run the evaluation of the corruptions with 10 repetitions 

In [12]:
num_repetitions = 10
results = evaluator.evaluate(model, num_repetitions, *corruptions)

0/20 (0.018346999999999447)
10/20 (0.1667909999999999)


### Investigate the impact on the predictive performance of the model

In [13]:
for validation_result in results:
    
    print(validation_result.corruption)
    print(f"""
     Score (AUC) on 
      clean data:     {validation_result.baseline_score}
      corrupted data: {np.mean(validation_result.corrupted_scores)}
     """)    
    print("\n")

MissingValues: {'column': 'marital_status', 'fraction': 0.99, 'sampling': 'MCAR', 'na_value': 'BROKEN'}

     Score (AUC) on 
      clean data:     0.8732229444875153
      corrupted data: 0.8026864879302179
     


MissingValues: {'column': 'age', 'fraction': 0.05, 'sampling': 'MCAR', 'na_value': -999}

     Score (AUC) on 
      clean data:     0.8732229444875153
      corrupted data: 0.8375170754321232
     


